In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup

In [ ]:
data = pd.read_csv('../datafiles/dataset_1_RE-BERT.csv')
data.info()

In [3]:
def pre_process(text):
  text = BeautifulSoup(str(text)).get_text()
  text = re.sub("[^a-zA-Z]", " ", text)
  text = text.lower()
  tokens = text.split()
  return tokens

In [ ]:
def generate_bio_tags(sentence, keywords):
    clean_content_tokens = pre_process(sentence)
    
    tags = ['O'] * len(clean_content_tokens)
    
    for keyword in keywords:
        keyword_tokens = pre_process(keyword)
        for i in range(len(clean_content_tokens)):
            if clean_content_tokens[i:i+len(keyword_tokens)] == keyword_tokens:
                tags[i] = 'B'
                for j in range(1, len(keyword_tokens)):
                    tags[i+j] = 'I'
                break
    
    return clean_content_tokens, tags

In [ ]:
tags_column=[]
clean_content=[]
for i, row in data.iterrows():
    keywords=[]
    # print(row['Feature (All Annotated)'])
    if row['Feature (All Annotated)'] is not None:
        keywords = str(row['Feature (All Annotated)']).split(';')
    clean_content_tokens, tag_sequence = generate_bio_tags(row['Sentence content'], keywords)
    tags_column.append(tag_sequence)
    clean_content.append(clean_content_tokens)

print(len(tags_column))
print(len(clean_content))

In [ ]:
new_df = pd.DataFrame({'App id': data['App id'].to_list(), 'clean_content': clean_content, 'tags': tags_column})
new_df.info()

In [ ]:
new_df.to_csv('../datafiles/dataset_1_preprocessed.csv')